In [1]:
import pandas
import sklearn
from sklearn import *

In [2]:
# small example
url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/EN/S08_09_Retail_Analytics/InputFeatures_Prob1.csv'
# large example, please outcomment if you want to try
# url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/EN/S08_09_Retail_Analytics/InputFeatures_Prob2.csv'

predDemand = pandas.read_csv(url)

# Dataset is now stored in a Pandas Dataframe predDemand
predDemand

,Unnamed: 0,avgPriceChoice,UPC,PRICE,PRICE_p2,FEATURE,DISPLAY,TPR_ONLY,RELPRICE
0,0,3.0,1600027528,2.5,6.25,0,0,0,0.833333
1,1,3.0,1600027528,3.0,9.00,0,0,0,1.000000
2,2,3.0,1600027528,3.5,12.25,0,0,0,1.166667
3,3,3.0,1600027564,2.5,6.25,0,0,0,0.833333
4,4,3.0,1600027564,3.0,9.00,0,0,0,1.000000
5,5,3.0,1600027564,3.5,12.25,0,0,0,1.166667
6,6,3.0,3000006340,2.5,6.25,0,0,0,0.833333
7,7,3.0,3000006340,3.0,9.00,0,0,0,1.000000
8,8,3.0,3000006340,3.5,12.25,0,0,0,1.166667
9,9,3.0,3800031829,2.5,6.25,0,0,0,0.833333


In [3]:
cwd = './'

In [4]:
import pickle

productList = predDemand['UPC'].unique()

regr = {}
for upc in productList:
    filename = cwd+str(upc)+'_demand_model.sav'
    print(upc)
    # load the model to disk
    regr[upc] = pickle.load(open(filename, 'rb'))

1600027528
1600027564
3000006340
3800031829


In [5]:
feature_list = ['PRICE', 'PRICE_p2', 'FEATURE', 'DISPLAY','TPR_ONLY','RELPRICE']

X = {}
y_pred = {}

# prepare blank series which will be added as a new column to the DataFrame predDemand
predictedValueSeries = pandas.Series()

for upc in productList:
  # Line 1 of for loop: load the data on the explanatory variable
  X[upc] = predDemand.loc[predDemand['UPC']==upc][feature_list]

  # Line 2: retrieve the UPC value
  upcIndex = predDemand.loc[predDemand['UPC']==upc].index

  # Line 3: predice the demands and make sure the demand is non-negative
  y_pred[upc] = regr[upc].predict(X[upc]).clip(0.0).round(1)

  # Line 4: add the predicted demand to the series
  predictedValueSeries = predictedValueSeries._append(pandas.Series(y_pred[upc], index = upcIndex))

predDemand['predictSales'] = predictedValueSeries
print(predDemand.to_string())

    Unnamed: 0  avgPriceChoice         UPC  PRICE  PRICE_p2  FEATURE  DISPLAY  TPR_ONLY  RELPRICE  predictSales
0            0             3.0  1600027528    2.5      6.25        0        0         0  0.833333          94.9
1            1             3.0  1600027528    3.0      9.00        0        0         0  1.000000          67.0
2            2             3.0  1600027528    3.5     12.25        0        0         0  1.166667          46.4
3            3             3.0  1600027564    2.5      6.25        0        0         0  0.833333          24.1
4            4             3.0  1600027564    3.0      9.00        0        0         0  1.000000          22.6
5            5             3.0  1600027564    3.5     12.25        0        0         0  1.166667          19.8
6            6             3.0  3000006340    2.5      6.25        0        0         0  0.833333           6.2
7            7             3.0  3000006340    3.0      9.00        0        0         0  1.000000       

In [6]:
# Please save it as 'predictedSales_Prob1.csv' if 'InputFeatures_Prob1.csv' is used
# Otherwise, please save it as 'predictedSales_Prob2.csv' if 'InputFeatures_Prob2.csv' is used
predDemand.to_csv(cwd +'predictedSales_Prob1.csv')